In [1]:
import sys
import os
import csv
import io
from google.oauth2 import service_account
from googleapiclient import discovery
from googleapiclient.http import MediaFileUpload
from datetime import datetime
import pandas as pd
import numpy as np
import unidecode
import warnings
import time
import pyodbc
print(os.path.dirname(sys.executable))

d:\data_sale\venv\Scripts


In [ ]:
server = '115.165.164.234'
driver = 'SQL Server'
db1 = 'PhaNam_eSales_PRO'
tcon = 'no'
uname = 'duyvq'
pword = '123VanQuangDuy'
cnxn = pyodbc.connect(driver='{ODBC Driver 17 for SQL Server}', 
                      host=server, database=db1, trusted_connection=tcon,
                      user=uname, password=pword)
cursor = cnxn.cursor()

In [ ]:
cursor.execute("SELECT @@version;") 
row = cursor.fetchone()
print(row[0])
cursor.close()
cnxn.close()

Microsoft SQL Server 2016 (SP2-GDR) (KB4505220) - 13.0.5101.9 (X64) 
	Jun 15 2019 23:15:58 
	Copyright (c) Microsoft Corporation
	Standard Edition (64-bit) on Windows Server 2016 Standard 10.0 <X64> (Build 14393: ) (Hypervisor)



In [ ]:
os.environ['COMPUTERNAME']

'DESKTOP-VA6QMGG'

In [ ]:
datenow = datetime.now().strftime("%Y%m%d")
param_1 = f"'20210831'"
param_2 = f"'{datenow}'"
query = f"EXEC [pr_OM_RawdataSellOutPayroll_BI] @Fromdate={param_1}, @Todate={param_2}"
df1 = pd.read_sql(query, cnxn)
cursor.close()
cnxn.close()

query1 = "EXEC [pr_OM_RawdataSellOutPayroll_BI] @Fromdate='20210801', @Todate='20210811'"
df1 = pd.read_sql(query2, cnxn)

In [ ]:
df1.columns

Index(['Mã Công Ty/CN', 'Công Ty/CN', 'Ngày Chứng Từ', 'Số Đơn Đặt Hàng',
       'Số Đơn Trả Hàng', 'Ngày Trả Hàng', 'Hóa Đơn', 'Ngày Tới Hạn TT',
       'Số Hợp Đồng', 'Trạng Thái', 'Mã KH Thuế', 'Tên KH Thuế',
       'Địa Chỉ KH Thuế', 'Mã Số Thuế', 'Mã KH DMS', 'Mã KH Cũ',
       'Tên Khách Hàng', 'Địa Chỉ KH', 'Mã Vùng BH', 'Tên Vùng BH',
       'Mã Khu Vực', 'Tên Khu Vực', 'Mã Tỉnh KH', 'Tên Tỉnh KH',
       'Mã Quận/HUyện', 'Tên Quận/HUyện', 'Phường/Xã', 'Mã Kênh KH',
       'Tên Kênh KH', 'Mã Kênh Phụ', 'Tên Kênh Phụ', 'Mã HCO', 'Tên HCO',
       'Mã Phân Loại HCO', 'Tên Phân Loại HCO', 'Mã Phân Hạng HCO',
       'Tên Phân Hạng HCO', 'Mã Sản Phẩm', 'Tên Sản Phẩm NB',
       'Tên Sản Phẩm Viết Tắt', 'Số Lô', 'Số Lượng', 'Đơn Giá (Có VAT)',
       'Doanh Số (Có VAT)', 'Đơn Giá (Chưa VAT)', 'Doanh Số (Chưa VAT)',
       'Ngày Đặt Đon', 'Người Tạo Đơn', 'Ngày Giao Hàng', 'Mã NV', 'Tên CVBH',
       'Tên Quản Lý TT', 'Tên Quản Lý Khu Vực', 'Tên Quản Lý Vùng', 'Mã NVGH',
       'Người

In [ ]:
df_rawdata_old = pd.read_csv('rawdata.csv', low_memory=False,  dayfirst=True, parse_dates=[' Ngày Chứng Từ'])

In [ ]:
columns = df_rawdata_old.columns
columns_list = []
for c in columns:
    c = c[1:]
    columns_list.append(c)

df_rawdata_old.columns = columns_list

In [ ]:
df = pd.concat([df1, df_rawdata_old])

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
df.shape

(196090, 77)

In [ ]:
df.replace(np.nan, '', inplace=True)
# df = df.head()
columns = df.columns
columns_list = []
for c in columns:
    c = unidecode.unidecode(c)
    columns_list.append(c)

df.columns = columns_list
df.columns = df.columns.str.replace(' ', '')
print(df.shape[0])

# df_product
url_product='https://drive.google.com/file/d/15ClbdtRJhqF6bIDNgRLqI1wAytIoPWWR/view?usp=sharing'
file_id_product=url_product.split('/')[-2]
dwn_url_product='https://drive.google.com/uc?id=' + file_id_product
df_product = pd.read_csv(dwn_url_product)
df = df.merge(df_product[[' Mã Sản Phẩm',' Ngành Hàng']], how = 'left',left_on='MaSanPham', right_on=' Mã Sản Phẩm',suffixes=('_left', '_right'), validate="m:1")
#df_dan_so
url_dan_so= 'https://drive.google.com/file/d/1bhb_Rh3Kpm8W0le7VY1TFB2soZvfryPS/view?usp=sharing'
file_id_dan_so=url_dan_so.split('/')[-2]
dwn_url_dan_so='https://drive.google.com/uc?id=' + file_id_dan_so
df_dan_so = pd.read_csv(dwn_url_dan_so)
df = df.merge(df_dan_so, how = 'left', left_on='TenTinhKH', right_on='TenTinhKH',suffixes=('_left', '_right'), validate="m:1")
#df_clean_data
df = df[df['MaSanPham'].str[:1] != 'V']
df = df[df['MaKenhKH'] != 'NB']
df['SDDH-CN'] =  df['SoDonDatHang']+'.'+df['MaCongTy/CN']
print(df.shape[0])
#Merap/Phanam
inlist = ['EH072', 'OH057', 'OH032', 'OH047','OH016','OH058','OH060','OH061',
          'OH062','OH063','OH064','OH065','OH070','OH069','OH066','OH068','OH067','EH105','OH071','OH079','OH081']
x = np.array(df['MaSanPham'])
condlist = [np.isin(x, inlist)]
choicelist = ['Pha Nam']
default = 'Merap'
df['Merap/PhaNam'] = np.select(condlist,choicelist, default=default)

#MaKenhKH2
x = np.array(df['Merap/PhaNam'])
condlist = [np.isin(x, ['Pha Nam'])]
choicelist = ['Pha Nam']
default = df['MaKenhKH']
df['MaKenhKH2'] = np.select(condlist,choicelist, default=default)

#MaMien(B/N)
x = np.array(df['MaVungBH'])
condlist = [np.isin(x, ['BB','HN'])]
choicelist = ['MB']
default = 'MN'
df['MaMien(B/N)'] = np.select(condlist,choicelist, default=default)

#VanPhong
df['VanPhong'] = df['CongTy/CN'].str.split(' ').apply(lambda x: x[-2:]).apply(lambda x: 'VP '+' '.join(x))

#df to csv
df.to_csv("raw_data_result.csv", index=False)

196600
196090


In [ ]:
df.columns

Index(['Ma Cong Ty/CN', 'Cong Ty/CN', 'Ngay Chung Tu', 'So Don Dat Hang',
       'So Don Tra Hang', 'Ngay Tra Hang', 'Hoa Don', 'Ngay Toi Han TT',
       'So Hop Dong', 'Trang Thai', 'Ma KH Thue', 'Ten KH Thue',
       'Dia Chi KH Thue', 'Ma So Thue', 'Ma KH DMS', 'Ma KH Cu',
       'Ten Khach Hang', 'Dia Chi KH', 'Ma Vung BH', 'Ten Vung BH',
       'Ma Khu Vuc', 'Ten Khu Vuc', 'Ma Tinh KH', 'Ten Tinh KH',
       'Ma Quan/HUyen', 'Ten Quan/HUyen', 'Phuong/Xa', 'Ma Kenh KH',
       'Ten Kenh KH', 'Ma Kenh Phu', 'Ten Kenh Phu', 'Ma HCO', 'Ten HCO',
       'Ma Phan Loai HCO', 'Ten Phan Loai HCO', 'Ma Phan Hang HCO',
       'Ten Phan Hang HCO', 'Ma San Pham', 'Ten San Pham NB',
       'Ten San Pham Viet Tat', 'So Lo', 'So Luong', 'Don Gia (Co VAT)',
       'Doanh So (Co VAT)', 'Don Gia (Chua VAT)', 'Doanh So (Chua VAT)',
       'Ngay Dat Don', 'Nguoi Tao Don', 'Ngay Giao Hang', 'Ma NV', 'Ten CVBH',
       'Ma NVGH', 'Nguoi Giao hang', 'Trang Thai Giao Hang', 'So Xuat Hang',
       'Don Vi 

In [2]:
# Google Sheet connection
scopes = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive',
          'https://www.googleapis.com/auth/drive.file']
jsonfile = 'D:/data_sale/datateam1599968716114-6f9f144b4262.json'
credentials = service_account.Credentials.from_service_account_file(jsonfile, scopes=scopes)
service = discovery.build('drive', 'v3', credentials=credentials)

In [ ]:
def datetime_to_int(dt):
    return int(dt.strftime("%Y%m%d%H%M%S"))

In [ ]:
#spreadsheets_id = '1qEwviiJcAtvWCLvc-5AXOgidaeHMsi2RWWIKiUehabI'
#https://developer.mozilla.org/en-US/docs/Web/HTTP/Basics_of_HTTP/MIME_types/Common_types
folder_id = '1nTlohjovIm7e2pf2LSHf3wGnyuuStGbh'
file_name = 'raw_data.csv'
file_type = 'text/csv'
file_body={'name':file_name,'addParents':[folder_id]}
query = f"parents = '{folder_id}'"
files = service.files().list(q=query).execute()
fileid  = files['files'][0]['id']

media = MediaFileUpload('{0}'.format(file_name), mimetype=file_type)
updated_file = service.files().update(fileId=fileid, body=file_body, media_body=media)
updated_file.execute()

{'kind': 'drive#file',
 'id': '1mwy3WmRATps0t36_kesuEY8Thbi4XgzW',
 'name': 'raw_data.csv',
 'mimeType': 'text/csv'}

In [16]:
#spreadsheets_id = '1qEwviiJcAtvWCLvc-5AXOgidaeHMsi2RWWIKiUehabI'
#https://developer.mozilla.org/en-US/docs/Web/HTTP/Basics_of_HTTP/MIME_types/Common_types
folder_id = '1s6-KRfQpFNTZINO0au9ETcEbqxfi_gp7'
file_name = 'output.xlsx'
file_type = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
file_body={'name':file_name,'addParents':[folder_id]}
query = f"parents = '{folder_id}'"
files = service.files().list(q=query).execute()
fileid  = files['files'][0]['id']
media = MediaFileUpload('{0}'.format(file_name), mimetype=file_type)
updated_file = service.files().update(fileId=fileid, body=file_body, media_body=media)
updated_file.execute()

In [18]:
fileid  = files['files'][1]['id']

In [21]:
# fileid

In [20]:
files

{'kind': 'drive#fileList',
 'incompleteSearch': False,
 'files': [{'kind': 'drive#file',
   'id': '1z_TbNPWG6Y01iwoQioAzMpBXruBHTSZC',
   'name': 'output.xlsx',
   'mimeType': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'},
  {'kind': 'drive#file',
   'id': '1OYwXwEt_wZbBK9uTjL1717z4YY838ZtL',
   'name': 'hoadondanhaptam.xlsx',
   'mimeType': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'}]}

In [ ]:
query = f"EXEC [pr_AR_TrackingDebtConfirm]  @Fromdate='20210801', @Todate='20210906'"
df1 = pd.read_sql(query, cnxn)
cursor.close()
cnxn.close()
print(df1.columns)

Index(['BranchID', 'SlsperID', 'PaymentsForm', 'Terms', 'OrdateDate',
       'CountOrdRelease', 'OrdAmtRelease', 'DeliveredOrder',
       'DeliveredOrderAmt', 'CountDebtConfirm', 'DebConfirmAmt',
       'CountDebtConfirmRelease', 'DebConfirmAmtRelease'],
      dtype='object')


In [ ]:
cursor.description

(('Mã Công Ty/CN', str, None, 30, 30, 0, False),
 ('Công Ty/CN', str, None, 200, 200, 0, False),
 ('Ngày Chứng Từ', datetime.datetime, None, 16, 16, 0, False),
 ('Số Đơn Đặt Hàng', str, None, 30, 30, 0, False),
 ('Số Đơn Trả Hàng', str, None, 30, 30, 0, False),
 ('Ngày Trả Hàng', str, None, 10, 10, 0, False),
 ('Hóa Đơn', str, None, 30, 30, 0, False),
 ('Ngày Tới Hạn TT', str, None, 10, 10, 0, False),
 ('Số Hợp Đồng', str, None, 50, 50, 0, False),
 ('Trạng Thái', str, None, 17, 17, 0, True),
 ('Mã KH Thuế', str, None, 30, 30, 0, False),
 ('Tên KH Thuế', str, None, 200, 200, 0, False),
 ('Địa Chỉ KH Thuế', str, None, 709, 709, 0, False),
 ('Mã Số Thuế', str, None, 200, 200, 0, False),
 ('Mã KH DMS', str, None, 50, 50, 0, False),
 ('Mã KH Cũ', str, None, 50, 50, 0, False),
 ('Tên Khách Hàng', str, None, 200, 200, 0, False),
 ('Địa Chỉ KH', str, None, 0, 0, 0, False),
 ('Mã Vùng BH', str, None, 10, 10, 0, False),
 ('Tên Vùng BH', str, None, 200, 200, 0, False),
 ('Mã Khu Vực', str, None, 

In [ ]:
with io.open(r'data.csv', 'w', encoding="utf-8", newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([x[0] for x in cursor.description])  # column headers
    for row in rows:
        writer.writerow(row)

In [ ]:
sys.getsizeof(csvfile)

208

In [ ]:
sys.getsizeof(writer)

80

In [ ]:
sys.getsizeof(rows)

128

In [ ]:
time.time()

1631084594.6773033

In [ ]:
query = f"EXEC [pr_AR_TrackingDebtConfirm]  @Fromdate='20161213', @Todate='20210920'"
query1 = f"EXEC [pr_OM_RawdataSellOutPayroll_BI] @Fromdate='20210501', @Todate='20210908'"

In [ ]:
start=time.time()
df1 = pd.read_sql(query, cnxn)
df1.to_csv('api_doanhthu_data.csv', index = False)
end=time.time()
duration=end-start
print(duration)

12.545311689376831


In [ ]:
sys.getsizeof(df1)

2498553

In [ ]:
start=time.time()
query1 = f"EXEC [pr_OM_RawdataSellOutPayroll_BI] @Fromdate='20210501', @Todate='20210908'"
rows = cursor.execute(query1)
with io.open(r'data.csv', 'w', encoding="utf-8", newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([x[0] for x in cursor.description])  # column headers
    for row in rows:
        writer.writerow(row)

end=time.time()
duration=end-start
print(duration)

206.98976111412048
